# 03_SymmetryAnalysis - Physics-SR Framework v4.1

## Stage 1.3: Symmetry Analysis (Power-law Detection)

**Author:** Zhengze Zhang  
**Affiliation:** Department of Statistics, Columbia University  
**Date:** January 2026  
**Version:** 4.1 (Structure-Guided Feature Library Enhancement + Computational Optimization)

---

### Purpose

Detect scale invariance and other physical symmetries to constrain the search space in symbolic regression. The primary method is power-law detection via log-log regression.

### Mathematical Foundation

**Power-law Relationship:**

If $y = C \cdot x_1^{\alpha_1} \cdot x_2^{\alpha_2} \cdots x_p^{\alpha_p}$, then taking logarithms:

$$\log(y) = \log(C) + \alpha_1 \log(x_1) + \alpha_2 \log(x_2) + \cdots + \alpha_p \log(x_p)$$

This is a linear regression problem in log-space. High $R^2$ indicates the relationship follows a power-law form.

### Implementation Note

Following Framework Section 8.2, we use the **simplified power-law detection** approach:
- Skip noise-sensitive Hessian-based separability tests
- Focus on robust log-log regression for power-law detection
- Extract exponent estimates for PySR initialization

### Reference

- Barenblatt, G. I. (1996). *Scaling, Self-Similarity, and Intermediate Asymptotics*. Cambridge University Press.
- Clauset, A., Shalizi, C. R., & Newman, M. E. (2009). Power-law distributions in empirical data. *SIAM Review*, 51(4), 661-703.

---
## Section 1: Header and Imports

In [ ]:
"""
03_SymmetryAnalysis.ipynb - Symmetry Analysis (Power-law Detection)
====================================================================

Three-Stage Physics-Informed Symbolic Regression Framework v4.1

This module provides:
- SymmetryAnalyzer: Detect power-law relationships via log-log regression
- Exponent estimation for PySR initialization
- Structural hints for constraining Stage 2 search

Algorithm (Simplified Power-law Detection):
    1. Filter valid data (positive values only)
    2. Apply log transformation: log(X), log(y)
    3. Fit multivariate linear regression in log space
    4. Compute R^2 to determine if power-law holds
    5. Extract exponent estimates as structural hints

Output Dictionary Keys (v4.1):
    - is_power_law: Boolean indicating power-law detection
    - estimated_exponents: Dict mapping feature names to exponents
    - power_law_r2: R-squared of log-log fit
    - structural_hints: Dict with hints for Stage 2
    - coefficient: Estimated multiplicative constant C
    - n_valid_samples: Number of samples used

Author: Zhengze Zhang
Affiliation: Department of Statistics, Columbia University
Contact: zz3239@columbia.edu
"""

# Import core module
%run 00_Core.ipynb

In [ ]:
# Additional imports for Symmetry Analysis
from sklearn.linear_model import LinearRegression
from typing import Dict, List, Tuple, Optional, Any

print("03_SymmetryAnalysis v4.1: Additional imports successful.")

---
## Section 2: Class Definition

In [ ]:
# ==============================================================================
# SYMMETRY ANALYZER CLASS
# ==============================================================================

class SymmetryAnalyzer:
    """
    Symmetry Analysis for Physics-Informed Symbolic Regression.
    
    This analyzer detects power-law relationships in data using log-log
    regression. Power-law detection is robust to noise (unlike Hessian-based
    separability tests) and provides valuable structural hints for PySR.
    
    The algorithm:
    1. Filters data to positive values (required for log transformation)
    2. Fits log(y) = log(C) + sum(alpha_i * log(x_i))
    3. Computes R^2 to determine if power-law structure exists
    4. Extracts exponent estimates as hints for Stage 2
    
    Attributes
    ----------
    r2_threshold : float
        Minimum R^2 to classify as power-law (default: 0.9)
    analysis_results : Optional[Dict]
        Stored results from analyze() call (public, v4.1)
    
    Methods
    -------
    analyze(X, y, feature_names) -> Dict
        Perform power-law detection
    get_structural_hints() -> Dict
        Get structural hints for Stage 2
    get_pysr_initial_population(n_equations) -> List[str]
        Generate initial equations for PySR
    print_symmetry_report() -> None
        Print detailed analysis report
    
    Examples
    --------
    >>> analyzer = SymmetryAnalyzer(r2_threshold=0.9)
    >>> result = analyzer.analyze(X, y, feature_names)
    >>> if result['is_power_law']:
    ...     print(f"Exponents: {result['estimated_exponents']}")
    """
    
    def __init__(
        self,
        r2_threshold: float = DEFAULT_POWERLAW_R2_THRESHOLD
    ):
        """
        Initialize SymmetryAnalyzer.
        
        Parameters
        ----------
        r2_threshold : float
            Minimum R^2 value to classify the relationship as power-law.
            Higher values are more conservative.
            Default: 0.9
        """
        self.r2_threshold = r2_threshold
        
        # Public attribute (v4.1 naming convention)
        self.analysis_results = None
        
        # Internal state (private, prefixed with underscore)
        self._feature_names = None
        self._is_power_law = False
        self._estimated_exponents = {}
        self._estimated_coefficient = None
        self._power_law_r2 = 0.0
        self._n_valid_samples = 0
        self._n_total_samples = 0
        self._log_intercept = None
        self._residual_std = None
        self._analysis_complete = False
    
    def analyze(
        self,
        X: np.ndarray,
        y: np.ndarray,
        feature_names: List[str]
    ) -> Dict[str, Any]:
        """
        Perform symmetry analysis (power-law detection).
        
        Parameters
        ----------
        X : np.ndarray
            Feature matrix of shape (n_samples, n_features)
        y : np.ndarray
            Target vector of shape (n_samples,)
        feature_names : List[str]
            Names of features corresponding to columns of X
        
        Returns
        -------
        Dict[str, Any]
            Dictionary containing (v4.1 keys):
            - is_power_law: bool, whether data follows power-law
            - estimated_exponents: Dict mapping feature names to exponents
            - power_law_r2: R-squared of log-log fit
            - structural_hints: Dict with hints for Stage 2
            - coefficient: Estimated multiplicative constant C
            - n_valid_samples: Number of samples used (positive values)
            - n_total_samples: Total number of input samples
            - r2_threshold: Threshold used for classification
        """
        self._feature_names = list(feature_names)
        self._n_total_samples = len(y)
        
        # Perform power-law detection
        self._is_power_law, self._estimated_exponents, self._power_law_r2 = \
            self._power_law_detection(X, y)
        
        self._analysis_complete = True
        
        # Build structural hints for Stage 2
        structural_hints = self._build_structural_hints()
        
        # Store results (public attribute, v4.1)
        self.analysis_results = {
            # v4.1 primary keys
            'is_power_law': self._is_power_law,
            'estimated_exponents': self._estimated_exponents,
            'power_law_r2': self._power_law_r2,
            'structural_hints': structural_hints,
            # Additional useful keys
            'coefficient': self._estimated_coefficient,
            'n_valid_samples': self._n_valid_samples,
            'n_total_samples': self._n_total_samples,
            'r2_threshold': self.r2_threshold,
            # Backward compatibility aliases
            'exponents': self._estimated_exponents,
            'r_squared': self._power_law_r2
        }
        
        return self.analysis_results
    
    def _power_law_detection(
        self,
        X: np.ndarray,
        y: np.ndarray
    ) -> Tuple[bool, Dict[str, float], float]:
        """
        Detect power-law relationship via log-log regression.
        
        If y = C * x1^a1 * x2^a2 * ... * xp^ap, then
        log(y) = log(C) + a1*log(x1) + a2*log(x2) + ... + ap*log(xp)
        
        Parameters
        ----------
        X : np.ndarray
            Feature matrix
        y : np.ndarray
            Target vector
        
        Returns
        -------
        Tuple[bool, Dict[str, float], float]
            - is_power_law: True if R^2 > threshold
            - estimated_exponents: Dict of estimated exponents
            - power_law_r2: R^2 of log-log fit
        """
        # Filter valid data (positive values only for log transformation)
        valid_mask = (y > 0) & np.all(X > 0, axis=1)
        self._n_valid_samples = np.sum(valid_mask)
        
        # Need minimum samples for regression
        min_samples = max(10, X.shape[1] + 2)
        if self._n_valid_samples < min_samples:
            return False, {name: 0.0 for name in self._feature_names}, 0.0
        
        X_valid = X[valid_mask]
        y_valid = y[valid_mask]
        
        # Log transformation
        log_X = np.log(X_valid)
        log_y = np.log(y_valid)
        
        # Fit linear regression in log space
        coefficients, intercept, r_squared = self._fit_log_log_regression(log_X, log_y)
        
        # Store results
        self._log_intercept = intercept
        self._estimated_coefficient = np.exp(intercept)
        
        # Compute residual standard deviation
        log_y_pred = log_X @ coefficients + intercept
        self._residual_std = np.std(log_y - log_y_pred)
        
        # Build exponents dictionary
        exponents = {
            name: float(coef) 
            for name, coef in zip(self._feature_names, coefficients)
        }
        
        # Determine if power-law
        is_power_law = r_squared > self.r2_threshold
        
        return is_power_law, exponents, r_squared
    
    def _fit_log_log_regression(
        self,
        log_X: np.ndarray,
        log_y: np.ndarray
    ) -> Tuple[np.ndarray, float, float]:
        """
        Fit linear regression in log-log space.
        
        Parameters
        ----------
        log_X : np.ndarray
            Log-transformed feature matrix
        log_y : np.ndarray
            Log-transformed target vector
        
        Returns
        -------
        Tuple[np.ndarray, float, float]
            - coefficients: Regression coefficients (exponents)
            - intercept: Regression intercept (log(C))
            - r_squared: R^2 of the fit
        """
        reg = LinearRegression()
        reg.fit(log_X, log_y)
        
        # Compute R^2
        log_y_pred = reg.predict(log_X)
        ss_res = np.sum((log_y - log_y_pred) ** 2)
        ss_tot = np.sum((log_y - np.mean(log_y)) ** 2)
        
        if ss_tot < EPS_DIV:
            r_squared = 0.0
        else:
            r_squared = 1.0 - ss_res / ss_tot
        
        return reg.coef_, reg.intercept_, r_squared
    
    def _build_structural_hints(
        self
    ) -> Dict[str, Any]:
        """
        Build structural hints for Stage 2.
        
        Returns
        -------
        Dict[str, Any]
            Structural hints including:
            - suggested_form: String description of suggested form
            - use_log_transform: Whether to consider log-transformed features
            - exponent_ranges: Suggested search ranges for exponents
            - active_variables: Variables with non-negligible exponents
        """
        hints = {
            'suggested_form': None,
            'use_log_transform': False,
            'exponent_ranges': {},
            'active_variables': [],
            'pysr_constraints': {}
        }
        
        if self._is_power_law:
            # Suggest power-law form
            terms = []
            for name, exp in self._estimated_exponents.items():
                if abs(exp) > 0.1:  # Non-negligible exponent
                    hints['active_variables'].append(name)
                    if abs(exp - round(exp)) < 0.1:
                        # Close to integer
                        int_exp = int(round(exp))
                        if int_exp == 1:
                            terms.append(name)
                        elif int_exp == -1:
                            terms.append(f"1/{name}")
                        else:
                            terms.append(f"{name}^{int_exp}")
                    else:
                        terms.append(f"{name}^{exp:.2f}")
                    
                    # Set exponent search range around estimated value
                    hints['exponent_ranges'][name] = (
                        exp - 1.0,
                        exp + 1.0
                    )
            
            if self._estimated_coefficient is not None:
                hints['suggested_form'] = f"{self._estimated_coefficient:.4f} * " + " * ".join(terms)
            else:
                hints['suggested_form'] = "C * " + " * ".join(terms)
            
            hints['use_log_transform'] = True
            
            # PySR constraints: suggest including power operators
            hints['pysr_constraints'] = {
                'binary_operators': ["+", "-", "*", "/", "^"],
                'unary_operators': ["exp", "log"],
                'complexity_of_operators': {
                    "^": 2,  # Power is relatively simple for power-law
                    "exp": 3,
                    "log": 3
                }
            }
        else:
            hints['suggested_form'] = "Non-power-law (try polynomial or other forms)"
            hints['pysr_constraints'] = {
                'binary_operators': ["+", "-", "*", "/"],
                'unary_operators': ["sin", "cos", "exp", "log"]
            }
        
        return hints
    
    def get_structural_hints(self) -> Dict[str, Any]:
        """
        Get structural hints for Stage 2.
        
        Returns
        -------
        Dict[str, Any]
            Structural hints dictionary
        
        Raises
        ------
        RuntimeError
            If analysis has not been performed
        """
        if not self._analysis_complete:
            raise RuntimeError("Must run analyze() before getting structural hints")
        return self._build_structural_hints()
    
    def get_pysr_initial_population(
        self,
        n_equations: int = 5
    ) -> List[str]:
        """
        Generate initial equation population for PySR based on detected structure.
        
        Parameters
        ----------
        n_equations : int
            Number of initial equations to generate
        
        Returns
        -------
        List[str]
            List of equation strings for PySR initialization
        
        Raises
        ------
        RuntimeError
            If analysis has not been performed
        """
        if not self._analysis_complete:
            raise RuntimeError("Must run analyze() before generating initial population")
        
        equations = []
        
        if self._is_power_law:
            # Generate power-law variations
            # Base equation
            terms = []
            for name, exp in self._estimated_exponents.items():
                if abs(exp) > 0.1:
                    terms.append(f"({name}^{exp:.2f})")
            
            if terms:
                # Exact estimated form
                base_eq = f"{self._estimated_coefficient:.4f} * " + " * ".join(terms)
                equations.append(base_eq)
                
                # Rounded exponent form
                terms_rounded = []
                for name, exp in self._estimated_exponents.items():
                    if abs(exp) > 0.1:
                        int_exp = int(round(exp))
                        terms_rounded.append(f"({name}^{int_exp})")
                
                if terms_rounded:
                    equations.append("C * " + " * ".join(terms_rounded))
                
                # Perturbed versions
                for i in range(min(n_equations - 2, 3)):
                    perturbed_terms = []
                    for name, exp in self._estimated_exponents.items():
                        if abs(exp) > 0.1:
                            perturbed_exp = exp + (i - 1) * 0.2
                            perturbed_terms.append(f"({name}^{perturbed_exp:.2f})")
                    if perturbed_terms:
                        equations.append("C * " + " * ".join(perturbed_terms))
        
        return equations[:n_equations]
    
    def print_symmetry_report(self) -> None:
        """
        Print a detailed symmetry analysis report in v4.1 format.
        """
        if not self._analysis_complete:
            print("Analysis not yet performed. Run analyze() first.")
            return
        
        print("=" * 70)
        print("=== Symmetry Analysis Results (Power-law Detection) ===")
        print("=" * 70)
        print()
        print("Analysis Configuration:")
        print(f"  R^2 threshold: {self.r2_threshold}")
        print(f"  Total samples: {self._n_total_samples}")
        print(f"  Valid samples (positive): {self._n_valid_samples}")
        print()
        print("-" * 70)
        print(" Power-law Detection:")
        print("-" * 70)
        print(f"  Log-log R-squared: {self._power_law_r2:.4f}")
        print(f"  Power-law detected: {'YES' if self._is_power_law else 'NO'}")
        print()
        
        if self._is_power_law:
            print("  Estimated Coefficient: C = {:.6f}".format(self._estimated_coefficient))
            print()
            print("  Estimated Exponents:")
            for name, exp in sorted(self._estimated_exponents.items(), key=lambda x: abs(x[1]), reverse=True):
                significance = "(significant)" if abs(exp) > 0.1 else "(negligible)"
                print(f"    {name}: {exp:+.4f} {significance}")
        
        print()
        print("-" * 70)
        print(" Structural Hints for Stage 2:")
        print("-" * 70)
        hints = self._build_structural_hints()
        print(f"  Suggested form: {hints['suggested_form']}")
        print(f"  Use log transform: {hints['use_log_transform']}")
        print(f"  Active variables: {hints['active_variables']}")
        print()
        print("=" * 70)

print("SymmetryAnalyzer class defined.")

---
## Section 3: Internal Tests

In [ ]:
# ==============================================================================
# TEST CONTROL FLAG
# ==============================================================================

_RUN_TESTS = False  # Set to True to run internal tests

if _RUN_TESTS:
    print("=" * 70)
    print(" RUNNING INTERNAL TESTS FOR 03_SymmetryAnalysis v4.1")
    print("=" * 70)

In [ ]:
# ==============================================================================
# TEST 1: Pure Power-law Data
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 1: Pure Power-law Data")
    
    # Generate true power-law: y = 0.5 * x1^2.5 * x2^(-1.5)
    np.random.seed(42)
    n_samples = 500
    
    x1 = np.random.uniform(0.5, 2.0, n_samples)
    x2 = np.random.uniform(0.5, 2.0, n_samples)
    
    C_true = 0.5
    alpha1_true = 2.5
    alpha2_true = -1.5
    
    y = C_true * (x1 ** alpha1_true) * (x2 ** alpha2_true)
    
    X = np.column_stack([x1, x2])
    feature_names = ['x1', 'x2']
    
    print(f"True equation: y = {C_true} * x1^{alpha1_true} * x2^{alpha2_true}")
    print()
    
    # Analyze
    analyzer = SymmetryAnalyzer(r2_threshold=0.9)
    result = analyzer.analyze(X, y, feature_names)
    
    # Verify output keys match v4.1 specification
    print("Output Dictionary Keys:")
    for key in result.keys():
        print(f"  {key}")
    print()
    
    print(f"Results:")
    print(f"  Power-law detected: {result['is_power_law']}")
    print(f"  R-squared: {result['power_law_r2']:.6f}")
    print(f"  Estimated coefficient: {result['coefficient']:.4f} (true: {C_true})")
    print()
    print("  Estimated exponents:")
    for name, exp in result['estimated_exponents'].items():
        true_exp = alpha1_true if name == 'x1' else alpha2_true
        error = abs(exp - true_exp) / abs(true_exp) * 100
        print(f"    {name}: {exp:.4f} (true: {true_exp}, error: {error:.2f}%)")
    
    # Verify
    if result['is_power_law'] and result['power_law_r2'] > 0.999:
        print("\n[PASS] Power-law correctly detected with high R^2")
    else:
        print(f"\n[WARNING] R^2 = {result['power_law_r2']:.6f}")

In [ ]:
# ==============================================================================
# TEST 2: Non-power-law Data
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 2: Non-power-law Data")
    
    # Generate non-power-law: y = sin(x1) + cos(x2)
    np.random.seed(42)
    n_samples = 500
    
    x1 = np.random.uniform(0.1, 3.0, n_samples)  # Positive for log
    x2 = np.random.uniform(0.1, 3.0, n_samples)
    
    y = np.sin(x1) + np.cos(x2) + 2  # Shift to ensure positive
    
    X = np.column_stack([x1, x2])
    feature_names = ['x1', 'x2']
    
    print(f"True equation: y = sin(x1) + cos(x2) + 2")
    print(f"This is NOT a power-law relationship")
    print()
    
    # Analyze
    analyzer = SymmetryAnalyzer(r2_threshold=0.9)
    result = analyzer.analyze(X, y, feature_names)
    
    print(f"Results:")
    print(f"  R-squared: {result['power_law_r2']:.4f}")
    print(f"  Power-law detected: {result['is_power_law']}")
    print()
    
    # Verification
    if not result['is_power_law']:
        print("  [PASS] Correctly identified as non-power-law")
    else:
        print("  [WARNING] Incorrectly classified as power-law")
        print(f"           (R^2 = {result['power_law_r2']:.4f} exceeds threshold)")

In [ ]:
# ==============================================================================
# TEST 3: Robustness to Noise
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 3: Robustness to Noise")
    
    # Test power-law detection at different noise levels
    np.random.seed(42)
    n_samples = 500
    
    # Generate base power-law data: y = x1^2 * x2^(-1)
    x1 = np.random.uniform(0.5, 2.0, n_samples)
    x2 = np.random.uniform(0.5, 2.0, n_samples)
    y_true = x1**2 * x2**(-1)
    
    noise_levels = [0.01, 0.05, 0.1, 0.2, 0.5]
    
    print(f"True equation: y = x1^2 * x2^(-1)")
    print(f"True exponents: x1=2.0, x2=-1.0")
    print()
    print(f"{'Noise Level':<15} {'R^2':<10} {'Detected':<12} {'x1 exp':<12} {'x2 exp':<12}")
    print("-" * 65)
    
    for noise in noise_levels:
        # Add multiplicative log-normal noise
        y_noisy = y_true * np.exp(noise * np.random.randn(n_samples))
        
        X = np.column_stack([x1, x2])
        
        analyzer = SymmetryAnalyzer(r2_threshold=0.9)
        result = analyzer.analyze(X, y_noisy, ['x1', 'x2'])
        
        detected = "YES" if result['is_power_law'] else "NO"
        x1_exp = result['estimated_exponents'].get('x1', 0)
        x2_exp = result['estimated_exponents'].get('x2', 0)
        
        print(f"{noise:<15.2f} {result['power_law_r2']:<10.4f} {detected:<12} {x1_exp:<12.3f} {x2_exp:<12.3f}")
    
    print()
    print("Note: Power-law detection via log-log regression is robust to")
    print("      moderate noise levels (up to ~10-20% multiplicative noise).")

In [ ]:
# ==============================================================================
# TEST 4: Integration with UserInputs
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 4: Integration with UserInputs")
    
    # Generate pendulum data (another power-law example)
    X, y, feature_names, user_inputs = generate_pendulum_data(
        n_samples=500, noise_level=0.01, seed=42
    )
    
    print(f"Pendulum data:")
    print(f"  Features: {feature_names}")
    print(f"  True equation: T = 2*pi*sqrt(L/g)")
    print(f"  Equivalent to: T = C * L^0.5 * g^(-0.5)")
    print(f"  Note: mass 'm' is irrelevant")
    print()
    
    # Run analysis
    analyzer = SymmetryAnalyzer(r2_threshold=0.9)
    result = analyzer.analyze(X, y, feature_names)
    
    print(f"Results:")
    print(f"  Power-law detected: {result['is_power_law']}")
    print(f"  R-squared: {result['power_law_r2']:.4f}")
    print()
    print("  Estimated exponents:")
    for name, exp in result['estimated_exponents'].items():
        expected = ""
        if name == 'L':
            expected = " (expected: 0.5)"
        elif name == 'g':
            expected = " (expected: -0.5)"
        elif name == 'm':
            expected = " (expected: ~0, irrelevant)"
        print(f"    {name}: {exp:.4f}{expected}")
    
    # Verify mass is detected as irrelevant
    m_exp = abs(result['estimated_exponents'].get('m', 0))
    if m_exp < 0.1:
        print("\n  [PASS] Mass correctly identified as irrelevant (exponent ~ 0)")
    else:
        print(f"\n  [WARNING] Mass exponent is {m_exp:.3f}, expected ~0")

In [ ]:
# ==============================================================================
# TEST 5: Full Report Output
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 5: Full Symmetry Report")
    
    # Generate warm rain data
    X, y, feature_names, _ = generate_warm_rain_data(
        n_samples=500, noise_level=0.01, seed=42
    )
    
    analyzer = SymmetryAnalyzer(r2_threshold=0.9)
    result = analyzer.analyze(X, y, feature_names)
    
    # Print full report
    analyzer.print_symmetry_report()

---
## Section 4: Module Summary

In [ ]:
# ==============================================================================
# MODULE SUMMARY
# ==============================================================================

print("=" * 70)
print(" 03_SymmetryAnalysis.ipynb v4.1 - Module Summary")
print("=" * 70)
print()
print("CLASS: SymmetryAnalyzer")
print("-" * 70)
print()
print("Purpose:")
print("  Detect power-law relationships via log-log regression.")
print("  Provides structural hints to constrain Stage 2 search.")
print()
print("Public Attributes (v4.1):")
print("  analysis_results  - Dict containing all analysis results")
print()
print("Main Methods:")
print("  analyze(X, y, feature_names)")
print("      Perform power-law detection")
print("      Returns: dict with is_power_law, estimated_exponents, etc.")
print()
print("  get_structural_hints()")
print("      Get structural hints for Stage 2")
print()
print("  get_pysr_initial_population(n_equations)")
print("      Generate initial equations for PySR based on detected structure")
print()
print("  print_symmetry_report()")
print("      Print detailed analysis report")
print()
print("Output Dictionary Keys (v4.1):")
print("  - is_power_law       : Boolean indicating power-law detection")
print("  - estimated_exponents: Dict mapping feature names to exponents")
print("  - power_law_r2       : R-squared of log-log fit")
print("  - structural_hints   : Dict with hints for Stage 2")
print("  - coefficient        : Estimated multiplicative constant")
print()
print("Usage Example:")
print("-" * 70)
print("""
# Create analyzer
analyzer = SymmetryAnalyzer(r2_threshold=0.9)

# Run analysis
result = analyzer.analyze(X, y, feature_names)

# Check for power-law (v4.1 keys)
if result['is_power_law']:
    print(f"Power-law detected with R^2 = {result['power_law_r2']:.4f}")
    print(f"Exponents: {result['estimated_exponents']}")
    
    # Get hints for PySR
    hints = result['structural_hints']
    print(f"Suggested form: {hints['suggested_form']}")
else:
    print("Non-power-law relationship detected")
""")
print()
print("=" * 70)
print("Module loaded successfully. Import via: %run 03_SymmetryAnalysis.ipynb")
print("=" * 70)